In [16]:
import pandas as pd
import datetime
import numpy as np
import math
import psycopg2
import psycopg2.extras

conn = psycopg2.connect(database="finances",
                        host="localhost",
                        user="postgres",
                        password="Karelia",
                        port=5432)

In [17]:
cursor = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

In [18]:
def get_shifts(username,year,month):
    command = """
    select start_time,end_time,employer from shifts 
    join contracts on contracts.contract_id = shifts.contract_id
    join users on contracts.user_id = contracts.user_id
    where 
    date_part('year',start_time) = %s and date_part('month',start_time) = %s or
    date_part('year',end_time) = %s and date_part('month',end_time) = %s
    and users.email = %s
    order by start_time asc;"""
    params = (year,month,year,month,username)
    cursor.execute(command,params)
    data = cursor.fetchall()
    return data

In [19]:
def merge_shifts(shifts,days,year,month):
    begin = pd.Timestamp(year=year,month=month,day=1,hour=0)
    end = begin + pd.Timedelta(1,unit="d")
    begin_f,end_f = begin.strftime("%Y-%d-%m %H:%S"),end.strftime("%Y-%d-%m %H:%S")
    shifts.loc[shifts['start_time'] < begin_f ,'start_time'] = begin_f
    shifts.loc[shifts['end_time'] >= end_f, 'end_time'] = end_f
    shifts["date"] = shifts["start_time"].dt.date.astype(str)
    shifts["start"] = shifts["start_time"].dt.time.astype(str).str.slice(start=0,stop=5)
    shifts["end"] = shifts["end_time"].dt.time.astype(str).str.slice(start=0,stop=5)
    calendar_f = pd.DataFrame(data={"date":days}).astype(str)
    merged = pd.merge(calendar_f, shifts.drop(columns=["start_time","end_time"]), on="date", how="outer")
    return merged

In [20]:
def create_calendar(year,month):
    normalized = pd.Timestamp(year=year,month=month,day=1)
    european_day_of_week = normalized.day_of_week
    if european_day_of_week != 6:
        first_cal_day = normalized - pd.Timedelta(european_day_of_week + 1, unit="d")
    else:
        first_cal_day = normalized
    last_day_of_month = pd.Timestamp(year=normalized.year,month=normalized.month,day=normalized.daysinmonth)
    day_diff = (last_day_of_month - first_cal_day).days * 1 + 1
    num_days = math.ceil(day_diff / 7) * 7
    last_cal_day =first_cal_day + pd.Timedelta(num_days-1, unit="d")
    days = pd.date_range(first_cal_day,last_cal_day)
    shift_data = get_shifts("koji.gabriel218@gmail.com",year,month)
    shifts = pd.DataFrame(shift_data)
    merged = merge_shifts(shifts,days,year,month)
    
    days = {}

    for row in merged.to_dict(orient="records"):
        if row["date"] not in days:
           days[row["date"]] = []
        
        if not pd.isnull(row["employer"]):
            shift = {"employer":row["employer"],"start":row["start"],"end":row["end"]}
            days[row["date"]].append(shift)
            
    calendar = [{"day":day,"shifts":days[day]} for day in days]
    return calendar

In [21]:
calendar = create_calendar(2024,5)

In [22]:
calendar

[{'day': '2024-04-28', 'shifts': []},
 {'day': '2024-04-29', 'shifts': []},
 {'day': '2024-04-30',
  'shifts': [{'employer': 'S-Market', 'start': '20:00', 'end': '00:00'}]},
 {'day': '2024-05-01', 'shifts': []},
 {'day': '2024-05-02', 'shifts': []},
 {'day': '2024-05-03', 'shifts': []},
 {'day': '2024-05-04', 'shifts': []},
 {'day': '2024-05-05', 'shifts': []},
 {'day': '2024-05-06', 'shifts': []},
 {'day': '2024-05-07', 'shifts': []},
 {'day': '2024-05-08', 'shifts': []},
 {'day': '2024-05-09', 'shifts': []},
 {'day': '2024-05-10', 'shifts': []},
 {'day': '2024-05-11', 'shifts': []},
 {'day': '2024-05-12', 'shifts': []},
 {'day': '2024-05-13', 'shifts': []},
 {'day': '2024-05-14',
  'shifts': [{'employer': 'Ikeaa', 'start': '08:00', 'end': '00:00'},
   {'employer': 'S-Market', 'start': '17:00', 'end': '00:00'}]},
 {'day': '2024-05-15', 'shifts': []},
 {'day': '2024-05-16', 'shifts': []},
 {'day': '2024-05-17', 'shifts': []},
 {'day': '2024-05-18', 'shifts': []},
 {'day': '2024-05-19',